In [ ]:
import sys
import os
import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

# import cell2location
# import scvi

from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42 # enables correct plotting of text for PDFs

In [ ]:
root_path = os.getcwd()

In [ ]:
results_folder = os.path.join(root_path, 'deconvolution')
# ref_run_name =  os.path.join(results_folder, 'reference_signatures') 
run_name = os.path.join(results_folder, 'RCTD')  

In [ ]:
run_name

In [ ]:
csv_files = [f for f in os.listdir(run_name) if f.endswith('.csv')]

In [ ]:
csv_files

In [ ]:
dfs = []

# Loop through the list of .csv files and read each file into a DataFrame
for file in csv_files:
    file_path = os.path.join(run_name, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

# Concatenate all the DataFrames
concatenated_df = pd.concat(dfs, ignore_index=True)
concatenated_df.set_index('spot_id_barcode', inplace = True)

In [ ]:
concatenated_df

In [ ]:
results_zonation_folder = os.path.join(root_path, 'analyzed', 'zonation')
file_names = [f for f in os.listdir(results_zonation_folder) if os.path.isfile(os.path.join(results_zonation_folder, f))]

adata_list = [ad.read(os.path.join(results_zonation_folder, file)) for file in file_names if file.endswith('.h5ad')]

In [ ]:
adata_list[1].obs

In [ ]:
# Combine anndata objects together
adata_concat = sc.concat(
    adata_list,
    label="library_id",
    uns_merge="unique",
    keys=[
        k
        for d in [adata.uns["spatial"] for adata in adata_list]
        for k, v in d.items()
    ],
    index_unique="-",
    join='outer',
)

In [ ]:
adata_concat.obs

In [ ]:
df_merged_zonation = pd.merge(concatenated_df, pd.DataFrame(adata_concat.obs[['Condition', 'Gender','zonation']]),  left_index=True, right_index=True)

In [ ]:
df_merged_zonation

In [ ]:
df_merged_zonation['all_conditions'] = df_merged_zonation['Gender'].astype(str) + '-' + df_merged_zonation['zonation'].astype(str) + '-' + df_merged_zonation['Condition'].astype(str)
df_merged_zonation['sex_treatment'] = df_merged_zonation['Gender'].astype(str) + '-'  + df_merged_zonation['Condition'].astype(str)
df_merged_zonation['sex_zonation'] = df_merged_zonation['Gender'].astype(str) + '-'  + df_merged_zonation['zonation'].astype(str)
df_merged_zonation['zonation_treatment'] = df_merged_zonation['zonation'].astype(str) + '-'  + df_merged_zonation['Condition'].astype(str)

In [ ]:
df_merged_zonation

In [ ]:
cell_types = df_merged_zonation.columns[3:32]

In [ ]:
# EDA: Plotting the distribution of a cell type across conditions
import seaborn as sns

for current_celltype in cell_types: 
    sns.boxplot(x='all_conditions', y=current_celltype, data=df_merged_zonation, showfliers=False)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
for current_celltype in cell_types: 
    sns.boxplot(x='Gender', y=current_celltype, data=df_merged_zonation, showfliers=False)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
for current_celltype in cell_types: 
    sns.boxplot(x='Condition', y=current_celltype, data=df_merged_zonation, showfliers=False)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
for current_celltype in cell_types: 
    sns.boxplot(x='zonation', y=current_celltype, data=df_merged_zonation, showfliers=False)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
for current_celltype in cell_types: 
    sns.boxplot(x='sex_treatment', y=current_celltype, data=df_merged_zonation, showfliers=False)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
for current_celltype in cell_types: 
    sns.boxplot(x='sex_zonation', y=current_celltype, data=df_merged_zonation, showfliers=False)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
for current_celltype in cell_types: 
    sns.boxplot(x='zonation_treatment', y=current_celltype, data=df_merged_zonation, showfliers=False)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
! jupyter nbconvert --to html 22c_Deconvolution_RCTD_Downstream.ipynb